## Урок 3.
# Построение модели классификации.

**План занятия**

* [Теоретическая часть](#theory)
    * [Балансировка классов](#class_balancing)
    * [Схемы оценки обобщающей способности алгоритма](#generalizing_ability)
    * [Обзор алгоритмов классификации](#classification_models)
        * [Логистическая регрессия](#logreg)
        * [Метод опорных векторов](#svm)
        * [k ближайших соседей](#knn)
        * [Случайный лес и бустинговые алгоритмы](#rb_and_gb)
* [Практическая часть](#practice)
    * [Загрузка данных](#load_data)
    * [Отбор признаков](#feature_selection)
    * [Нормализация данных](#normalization)
    * [Разбиение на train и test](#train_and_test)
    * [Балансировка целевой переменной](#target_balancing)
    * [Сохранение обучающего и тестового датасетов](#train_test_saving)
    * [Построение и оценка базовых моделей](#baseline_modeling)
    * [Выбор лучшей модели и подбор гиперпараметров](#tuning_best_model)

## <center>Теоретическая часть<a class="anchor" id="theory"></a><center>

### Балансировка классов<a class="anchor" id="class_balancing"></a>

**Описание проблемы**

Для многих алгоритмов классификации существенной проблемой является дисбаланс классов целевой переменной. В случаях когда примеров одного класса существенно больше, чем примеров другого класса, возникает вероятность того, что алгоритм переучиться на примерах превалирующего класса. Ситуация ещё больше усугубляется при многоклассовой классификации. 

**Способы борьбы с дисбалансом классов**

1. Собрать больше данных
2. Выбрать подходящую метрику качества
3. Попробовать разные модели, одни модели более устойчивы к несбалансированным данным, чем другие
4. Штраф за ошибки при прогнозе меньшего класса
5. Undersampling и Oversampling
6. Создание синтетических примеров для меньшего класса

<img src="balancing.png">

### Схемы оценки обобщающей способности алгоритма<a class="anchor" id="generalizing_ability"></a>

**Обобщающая способность алгоритма**

Модель, обладающая хорошей обобщающей способностью, способна предсказывать примерно на одном и том же уровне качества, как на обучающем датасете, так и на новых данных.

В процессе построения модели, нам необходимо оценивать качество прогнозов на данных, которые модель не видела во время обучения, чтобы избежать недообучения / переобучения нашего алгоритма. Мы поговорим подробнее об этих явлениях на следующем занятии. 

Для эмитации прогнозирования на новых данных, имея в распоряжении только обучающую выборку, используют два приёма - *отложенную выборку* и *кросс-валидацию*.

**Отложенная выборка**

Обучающая выборка разбивается на две части train и test, в тестовую часть, как правило, идёт от 20% до 33% наблюдений, остальное в обучающую. На обучающей части мы тренируем нашу модель и настраиваем гиперпараметры, а после сравниваем прогнозы между обучающей и тестовой частями, что бы понять, как модель поведёт себя на данных, которые не видела до этого.

**Кросс-валидация**

Данный способ, в своей основе, похож на предыдущий. Обучающая выборка разбивается на k непересекающихся частей, как правило 3 или 5, после чего каждая из частей поочерёдно выступает в роли тестовой части, а оставшиеся в роли обучающей. Таким образом модель проходи k стадий проверок, полностью задействуя обучающий датасет.

<img src="cross_val.png">

### Обзор алгоритмов классификации<a class="anchor" id="classification_models"></a>

**Логистическая регрессия**<a class="anchor" id="logreg"></a>

<img src="log_reg.png">

**Метод опорных векторов**<a class="anchor" id="svm"></a>

<img src="svm.png">

**k ближайших соседей**<a class="anchor" id="knn"></a>

<img src="knn.png">

**Случайный лес (бэгинг) и бустинговые алгоритмы**<a class="anchor" id="rb_and_gb"></a>

<img src="rf.jpg">

## <center>Практическая часть<a class="anchor" id="practice"></a><center>

**Подключение библиотек и скриптов**

In [0]:
# !pip install xgboost
# !pip install lightgbm
# !pip install catboost

In [0]:
import pandas as pd
import numpy as np
import pickle
import random

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
import warnings
warnings.simplefilter('ignore')

In [0]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [0]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1) 

**Пути к директориям и файлам**

In [0]:
# input
DATASET_PATH = '../training_project_data.csv'
PREP_DATASET_PATH = '../training_project_data_prep.csv'

# output
TRAIN_FULL_PATH = '../training_project_train_full.csv'
TRAIN_PART_PATH = '../training_project_train_part_b.csv'
TEST_PART_PATH = '../training_project_test_part.csv'

SCALER_FILE_PATH = '../scaler.pkl'

### Загрузка данных<a class="anchor" id="load_data"></a>

**Описание базового датасета**

* **LIMIT_BAL** - Сумма предоставленного кредита
* **SEX** - Пол (1=мужчина, 2=женщина)
* **EDUCATION** - Образование (1=аспирантура, 2=университет, 3=старшая школа, 4=прочее, 5=неизвестно, 6=неизвестно)
* **MARRIAGE** - Семейное положение (1=женат/замужен, 2=не женат/не замуженм, 3=прочее)
* **AGE** - Возраст (в годах)
* **PAY_1** - Статус погашения в Сентябре (-1=погашен полностью, 0=погашен частично, 1=отсрочка платежа на один месяц, .., 3=отсрочка платежа на три месяца и более)
* **PAY_2** - Статус погашения в Августе
* **PAY_3** - Статус погашения в Июле
* **PAY_4** - Статус погашения в Июне
* **PAY_5** - Статус погашения в Мае
* **PAY_6** - Статус погашения в Апреле
* **BILL_AMT1** - Сумма выписки по счету в Сентябре
* **BILL_AMT2** - Сумма выписки по счету в Августе
* **BILL_AMT3** - Сумма выписки по счету в Июле
* **BILL_AMT4** - Сумма выписки по счету в Июне
* **BILL_AMT5** - Сумма выписки по счету в Мае
* **BILL_AMT6** - Сумма выписки по счету в Апреле
* **PAY_AMT1** - Сумма предыдущего платежа в Сентябре
* **PAY_AMT2** - Сумма предыдущего платежа в Августе
* **PAY_AMT3** - Сумма предыдущего платежа в Июле
* **PAY_AMT4** - Сумма предыдущего платежа в Июне
* **PAY_AMT5** - Сумма предыдущего платежа в Мае
* **PAY_AMT6** - Сумма предыдущего платежа в Апреле
* **NEXT_MONTH_DEFAULT** - Просрочка платежа в следующем месяце (1=да, 0=нет)

In [0]:
df_base = pd.read_csv(DATASET_PATH)
df = pd.read_csv(PREP_DATASET_PATH)

df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,PAY_4_2,PAY_4_3,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3
0,150000.0,2,2,2,24,1,2,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,0,1,0,0,1,0,0,1,0,0
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,0,0,0,1,0,0,0,1,0,0
3,150000.0,2,2,2,35,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,70000.0,2,1,1,35,1,2,2,2,2,...,1,0,0,0,1,0,0,1,0,0


**Выделение целевой переменной и групп признаков**

In [0]:
TARGET_NAME = 'NEXT_MONTH_DEFAULT'
BASE_FEATURE_NAMES = df_base.columns.drop(TARGET_NAME).tolist()
NEW_FEATURE_NAMES = df.columns.drop([TARGET_NAME, 'ID'] + BASE_FEATURE_NAMES).tolist()

### Отбор признаков<a class="anchor" id="feature_selection"></a>

In [0]:
NUM_FEATURE_NAMES = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                     'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

CAT_FEATURE_NAMES = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

SELECTED_FEATURE_NAMES = NUM_FEATURE_NAMES + NEW_FEATURE_NAMES

### Приведение типов для модели CatBoost

In [0]:
for colname in CAT_FEATURE_NAMES:
    df[colname] = pd.Categorical(df[colname])
    
df[CAT_FEATURE_NAMES].dtypes

SEX          category
EDUCATION    category
MARRIAGE     category
PAY_1        category
PAY_2        category
PAY_3        category
PAY_4        category
PAY_5        category
PAY_6        category
dtype: object

### Нормализация данных<a class="anchor" id="normalization"></a>

In [0]:
scaler = StandardScaler()

df_norm = df.copy()
df_norm[NUM_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUM_FEATURE_NAMES])

df = df_norm.copy()

**Сохранение модели для нормализации данных**

In [0]:
with open(SCALER_FILE_PATH, 'wb') as file:
    pickle.dump(scaler, file)

### Разбиение на train и test<a class="anchor" id="train_and_test"></a>

In [0]:
X = df[SELECTED_FEATURE_NAMES]
y = df[TARGET_NAME]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.30, random_state=21)

### Балансировка целевой переменной<a class="anchor" id="target_balancing"></a>

In [0]:
df_for_balancing = pd.concat([X_train, y_train], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, TARGET_NAME)
    
df_balanced[TARGET_NAME].value_counts()

0    5500
1    4500
Name: NEXT_MONTH_DEFAULT, dtype: int64

In [0]:
X_train = df_balanced.drop(columns=TARGET_NAME)
y_train = df_balanced[TARGET_NAME]

### Сохранение обучающего и тестового датасетов<a class="anchor" id="train_test_saving"></a> 

In [0]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [0]:
df.to_csv(TRAIN_FULL_PATH, index=False, encoding='utf-8')
train.to_csv(TRAIN_PART_PATH, index=False, encoding='utf-8')
test.to_csv(TEST_PART_PATH, index=False, encoding='utf-8')

### Построение и оценка базовых моделей<a class="anchor" id="baseline_modeling"></a> 

**Логистическая регрессия**

In [0]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.70      0.87      0.78      5500
           1       0.77      0.54      0.64      4500

   micro avg       0.72      0.72      0.72     10000
   macro avg       0.74      0.71      0.71     10000
weighted avg       0.73      0.72      0.71     10000

TEST

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      2305
           1       0.53      0.53      0.53       695

   micro avg       0.78      0.78      0.78      3000
   macro avg       0.69      0.69      0.69      3000
weighted avg       0.78      0.78      0.78      3000

CONFUSION MATRIX

col_0                  0    1
NEXT_MONTH_DEFAULT           
0                   1978  327
1                    329  366


**k ближайших соседей**

In [0]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)

y_train_pred = model_knn.predict(X_train)
y_test_pred = model_knn.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.93      0.76      0.84      5500
           1       0.76      0.93      0.84      4500

   micro avg       0.84      0.84      0.84     10000
   macro avg       0.85      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000

TEST

              precision    recall  f1-score   support

           0       0.85      0.67      0.75      2305
           1       0.36      0.60      0.45       695

   micro avg       0.66      0.66      0.66      3000
   macro avg       0.60      0.64      0.60      3000
weighted avg       0.73      0.66      0.68      3000

CONFUSION MATRIX

col_0                  0    1
NEXT_MONTH_DEFAULT           
0                   1552  753
1                    278  417


**Бустинговые алгоритмы**

*XGBoost*

In [0]:
model_xgb = xgb.XGBClassifier(random_state=21)
model_xgb.fit(X_train, y_train)

y_train_pred = model_xgb.predict(X_train)
y_test_pred = model_xgb.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.74      0.86      0.79      5500
           1       0.78      0.63      0.70      4500

   micro avg       0.75      0.75      0.75     10000
   macro avg       0.76      0.74      0.74     10000
weighted avg       0.76      0.75      0.75     10000

TEST

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      2305
           1       0.50      0.55      0.53       695

   micro avg       0.77      0.77      0.77      3000
   macro avg       0.68      0.69      0.69      3000
weighted avg       0.78      0.77      0.77      3000

CONFUSION MATRIX

col_0                  0    1
NEXT_MONTH_DEFAULT           
0                   1928  377
1                    311  384


*LightGBM*

In [0]:
model_lgbm = lgbm.LGBMClassifier(random_state=21)
model_lgbm.fit(X_train, y_train)

y_train_pred = model_lgbm.predict(X_train)
y_test_pred = model_lgbm.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      5500
           1       0.89      0.85      0.87      4500

   micro avg       0.89      0.89      0.89     10000
   macro avg       0.89      0.88      0.88     10000
weighted avg       0.89      0.89      0.88     10000

TEST

              precision    recall  f1-score   support

           0       0.86      0.85      0.86      2305
           1       0.53      0.54      0.53       695

   micro avg       0.78      0.78      0.78      3000
   macro avg       0.69      0.69      0.69      3000
weighted avg       0.78      0.78      0.78      3000

CONFUSION MATRIX

col_0                  0    1
NEXT_MONTH_DEFAULT           
0                   1970  335
1                    323  372


*CatBoost*

In [0]:
model_catb = catb.CatBoostClassifier(silent=True, random_state=21)
model_catb.fit(X_train, y_train)

y_train_pred = model_catb.predict(X_train)
y_test_pred = model_catb.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      5500
           1       0.85      0.77      0.81      4500

   micro avg       0.84      0.84      0.84     10000
   macro avg       0.84      0.83      0.83     10000
weighted avg       0.84      0.84      0.84     10000

TEST

              precision    recall  f1-score   support

           0       0.86      0.85      0.86      2305
           1       0.53      0.55      0.54       695

   micro avg       0.78      0.78      0.78      3000
   macro avg       0.69      0.70      0.70      3000
weighted avg       0.79      0.78      0.78      3000

CONFUSION MATRIX

col_0                0.0  1.0
NEXT_MONTH_DEFAULT           
0                   1960  345
1                    311  384


### Выбор лучшей модели и подбор гиперпараметров<a class="anchor" id="tuning_best_model"></a> 

In [0]:
model_catb = catb.CatBoostClassifier(class_weights=[1, 3.5], silent=True, random_state=21)

**Подбор гиперпараметров**

In [0]:
params = {'n_estimators':[50, 100, 200, 500, 700, 1000, 1200, 1500],
          'max_depth':[3, 5, 7]}

In [0]:
cv=KFold(n_splits=3, random_state=21, shuffle=True)

In [0]:
%%time

rs = RandomizedSearchCV(model_catb, params, scoring='f1', cv=cv, n_jobs=-1)
rs.fit(X, y)

Wall time: 8min


In [0]:
rs.best_params_

{'n_estimators': 500, 'max_depth': 3}

In [0]:
rs.best_score_

0.5404695069816147

**Обучение и оценка финальной модели**

In [0]:
%%time

final_model = catb.CatBoostClassifier(n_estimators=500, max_depth=3,
                                      silent=True, random_state=21)
final_model.fit(X_train, y_train)

y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.75      0.85      0.79      5500
           1       0.78      0.64      0.71      4500

   micro avg       0.76      0.76      0.76     10000
   macro avg       0.76      0.75      0.75     10000
weighted avg       0.76      0.76      0.75     10000

TEST

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      2305
           1       0.51      0.57      0.54       695

   micro avg       0.77      0.77      0.77      3000
   macro avg       0.69      0.70      0.69      3000
weighted avg       0.78      0.77      0.78      3000

CONFUSION MATRIX

col_0                0.0  1.0
NEXT_MONTH_DEFAULT           
0                   1919  386
1                    296  399
Wall time: 10 s
